In [92]:
import numpy as np
import sqlite3
import pandas as pd
import csv
from datetime import datetime, timedelta
import time
import datetime

In [127]:
save_list=np.load("save_list.npy")
now_list=np.load("now.npy")

In [93]:
df = pd.read_csv('公司/上市.csv')  
a=df["有價證券代號及名稱"].tolist()
df = pd.read_csv('公司/上櫃.csv')  
b=df["有價證券代號及名稱"].tolist()
df = pd.read_csv('公司/興櫃.csv')  
c=df["有價證券代號及名稱"].tolist()



In [117]:
#查詢list in str
#any('1101' in item for item in a)
#查詢list in str
def search_name(name):
    company=[]
    
    for i in a:
        if i.__contains__(name) :
            company=i
            #print(i, " is containing")
    for i in b:
        if i.__contains__(name) :
            company=i
            #print(i, " is containing")
    for i in c:
        if i.__contains__(name) :
            company=i
            #print(i, " is containing")
    return company

In [123]:
db =  sqlite3.connect('db.sqlite3')
#db.execute("INSERT INTO type_data (tag)   VALUES ('{}')".format(data))
db.execute("delete from basicHistory")
db.commit()
db =  sqlite3.connect('db.sqlite3')
#db.execute("INSERT INTO type_data (tag)   VALUES ('{}')".format(data))
db.execute("delete from technicHistory")
db.commit()

In [124]:
#基本面歷史
pk=1
for i in range(len(save_list)):
    name=search_name(save_list[i][0]).replace("\u3000", " ")
    start_date=save_list[i][1]
    over_date=save_list[i][2]
    buy_price=save_list[i][3]
    sell_price=save_list[i][4]
    return_value=(float(sell_price)-float(buy_price))/float(buy_price)
    return_value= round(return_value*100,2)
    if return_value>0:
        types="+"
    else:
        types="-"

    db =  sqlite3.connect('db.sqlite3')
    db.execute("INSERT INTO basicHistory (id,stock_name,start_date,buy_price,over_date,sell_price,return_value,type)   VALUES ('{}','{}','{}','{}','{}','{}','{}','{}')".format(pk,name,start_date,buy_price,over_date,sell_price,return_value,types))
    db.commit()
    db.close()
    pk=pk+1

In [126]:
#技術面歷史
pk=1
for i in range(len(save_list)):
    name=search_name(save_list[i][0]).replace("\u3000", " ")
    start_date=save_list[i][1]
    over_date=save_list[i][2]
    buy_price=save_list[i][3]
    sell_price=save_list[i][4]
    return_value=(float(sell_price)-float(buy_price))/float(buy_price)
    return_value= round(return_value*100,2)
    if return_value>0:
        types="+"
    else:
        types="-"

    db =  sqlite3.connect('db.sqlite3')
    db.execute("INSERT INTO technicHistory (id,stock_name,start_date,buy_price,over_date,sell_price,return_value,type)   VALUES ('{}','{}','{}','{}','{}','{}','{}','{}')".format(pk,name,start_date,buy_price,over_date,sell_price,return_value,types))
    db.commit()
    db.close()
    pk=pk+1

In [141]:
db =  sqlite3.connect('db.sqlite3')
#db.execute("INSERT INTO type_data (tag)   VALUES ('{}')".format(data))
db.execute("delete from basicCurrent")
db.commit()
db =  sqlite3.connect('db.sqlite3')
#db.execute("INSERT INTO type_data (tag)   VALUES ('{}')".format(data))
db.execute("delete from technicCurrent")
db.commit()

In [142]:
#基本面現在
pk=1
for i in range(len(now_list)):
    final_update="2020-07-14"
    name=search_name(now_list[i][0]).replace("\u3000", " ")
    start_date="2020-07-10"
    start_price=now_list[i][2]
    over_date="2020-08-14"
    current_price=now_list[i][3]
    now_return=now_list[i][1]
    if return_value>0:
        types="+"
    else:
        types="-"
        
        
    db =  sqlite3.connect('db.sqlite3')
    db.execute("INSERT INTO basicCurrent (id,final_update,stock_name,start_date,start_price,over_date,current_price,now_return,type)   VALUES ('{}','{}','{}','{}','{}','{}','{}','{}','{}')".format(pk,final_update,name,start_date,start_price,over_date,current_price,now_return,types))
    db.commit()
    db.close()
    pk=pk+1

In [143]:
#基本面現在
pk=1
for i in range(len(now_list)):
    final_update="2020-07-14"
    name=search_name(now_list[i][0]).replace("\u3000", " ")
    start_date="2020-07-10"
    start_price=now_list[i][2]
    over_date="2020-08-14"
    current_price=now_list[i][3]
    now_return=now_list[i][1]
    if return_value>0:
        types="+"
    else:
        types="-"
        
        
    db =  sqlite3.connect('db.sqlite3')
    db.execute("INSERT INTO technicCurrent (id,final_update,stock_name,start_date,start_price,over_date,current_price,now_return,type)   VALUES ('{}','{}','{}','{}','{}','{}','{}','{}','{}')".format(pk,final_update,name,start_date,start_price,over_date,current_price,now_return,types))
    db.commit()
    db.close()
    pk=pk+1